<a href="https://colab.research.google.com/github/sssylvia0/Uplift_Model/blob/main/Model_Nov.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Uplift and Profit prediction

1. Model (Traning Data: data202306.csv)

*  **Uplift**:  CasualML_uplift_202306_1-5.pkl

* **X-Learner**:  'EconML_X_202306_new_1.pkl', 'Value_EconML_X_202306.pkl',

*   **Profit**:  Econml_RF_202306.pkl, Econml_CF_202306.pkl

2. Time：約20分鐘
   
   需要手動重啟!





In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
import zipfile

# Specify the path to the zip file
#zip_file_path = 'data202305.csv.zip'
zip_file_path = '/content/drive/MyDrive/Sinopac/Rawdata/data202311.csv.zip'


# Unzip the folder
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall('/content/')
print("Unzipped successfully.")

Unzipped successfully.


In [8]:
import pandas as pd
import numpy as np

# Load your May data
oct_data = pd.read_csv('/content/data202311.csv', index_col=0)

unino = oct_data['UNINO']
oct_data.drop(['換匯發送名單','TOTALNTAMOUNT', '跨售行為', 'UNINO'], axis = 'columns', inplace = True)

#201 columns
oct_data.columns

Index(['年齡', '公司戶記號', '性別', '呆帳記號_含信用卡', '客戶年收入', '網路會員記號', '網路使用記號_行動銀行',
       '小mma記號', '網路服務記號', '網路使用記號',
       ...
       '客戶行業別_醫療保健', '客戶行業別_金融業', '客戶行業別_電子業', '對帳單寄送記號_0', '對帳單寄送記號_1',
       '對帳單寄送記號_2', '信用卡VIP客戶註記_Y', '外幣存款記號_Y', '信用卡轉呆記號_Y', '信用卡自動扣款記號_Y'],
      dtype='object', length=201)

## CausalML

In [2]:
!pip install causalml

In [1]:
import importlib
print(importlib.metadata.version('scikit-learn') )

1.0.2


In [3]:
import numpy as np
import pandas as pd
import duecredit

from causalml.inference.tree import UpliftRandomForestClassifier
from causalml.metrics import plot_gain, get_cumgain
from sklearn.model_selection import train_test_split

import importlib
print(importlib.metadata.version('causalml') )

/usr/local/lib/python3.10/dist-packages/shap/utils/_clustering.py:35: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def _pt_shuffle_rec(i, indexes, index_mask, partition_tree, M, pos):
/usr/local/lib/python3.10/dist-packages/shap/utils/_clustering.py:54: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def delta_minimization_order(all_masks, max_swap_size=100, num_

0.14.1


In [ ]:
%%time
import pickle
def load_model(model_path):
   """Load a pickled model."""
   return pickle.load(open(model_path, "rb"))

def get_results(model_index, df, data, model_prefix="CasualML_uplift_202306_"):
   """Run model predictions and update DataFrame."""
   model_path = f"/content/drive/MyDrive/Model/{model_prefix}{model_index}.pkl"
   model = load_model(model_path)
   df_res = model.predict(data, full_output=True)
   df[f'Uplift_{model_index}|T=0'] = df_res.iloc[:, 0]
   df[f'Uplift_{model_index}|T=1'] = df_res.iloc[:, 1]
   print(f'Finish running uplift_new_{model_index}')
   return df

def calculate_mean_uplifts(res, model_count=5):
   """Calculate mean uplifts and return a DataFrame."""
   T = [f'Uplift_{i}|T=0' for i in range(1, model_count + 1)]
   T1 = [f'Uplift_{i}|T=1' for i in range(1, model_count + 1)]
   res_columns = res[T + T1]
   pred = pd.DataFrame()
   pred['Uplift_mean|T=0'] = res_columns[T].mean(axis=1)
   pred['Uplift_mean|T=1'] = res_columns[T1].mean(axis=1)
   pred['Uplift_mean'] = pred['Uplift_mean|T=1'] - pred['Uplift_mean|T=0']
   return pred

# Example usage
res_df = pd.DataFrame()
for i in range(1, 6):
   res_df = get_results(i, res_df, oct_data)
pred = calculate_mean_uplifts(res_df)
pred

In [ ]:
#暫存causalml的結果
pred.to_csv('temp.csv')

##X-Learner

In [ ]:
!pip install scikit-learn==1.2.2
!pip install --upgrade DecisionTreeClassifier

In [4]:
#避免版本衝突，重新下載之後需要手動重啟

In [ ]:
import importlib
print(importlib.metadata.version('scikit-learn') )

In [ ]:
import pandas as pd
import numpy as np

# Load your May data
oct_data = pd.read_csv('/content/data202311.csv', index_col=0)

unino = oct_data['UNINO']
oct_data.drop(['換匯發送名單','TOTALNTAMOUNT', '跨售行為', 'UNINO'], axis = 'columns', inplace = True)

#201 columns
oct_data.columns

In [ ]:
!pip install econml

In [ ]:
# Main imports
from econml.metalearners import XLearner

# Helper imports
import numpy as np
from numpy.random import binomial, multivariate_normal, normal, uniform
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, GradientBoostingRegressor
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
%%time
import pickle
import pandas as pd

def get_other_results(model_list, name_list, df, data):
  for i in range(len(model_list)):
    with open ("/content/drive/MyDrive/Model/"+ model_list[i], "rb") as file:
      est = pickle.load(file)
    if i ==2:
      df[name_list[i]] = est.predict(data)
    else :
      df[name_list[i]] = est.effect(data)
  return df

##使用
pred = pd.read_csv('temp.csv', index_col = 0)
pred.insert(0, '換匯基因標籤', oct_data['換匯基因標籤'].values)
pred.insert(0, 'UNINO', unino.values)
model_list = ['EconML_X_202306_new_1.pkl', 'Value_EconML_X_202306.pkl', 'Econml_RF_202306_new.pkl', 'Econml_CF_202306_new.pkl']
name_list = ['X_te_2', 'Value_X', 'E(V)|T = 0, D = 1', 'te']
pred = get_other_results(model_list, name_list, pred, oct_data)
pred['E(V)|T = 1, D = 1'] = pred['E(V)|T = 0, D = 1'] + pred['te']
pred
#pred.to_csv('/content/drive/MyDrive/Sinopac/Results/Prediction_results_202311.csv')
#1066146 rows × 10 columns

In [ ]:
pred.describe()

In [ ]:
res = pd.DataFrame()
res = pred.iloc[:,:7]
res['X_te_2'] = pred['X_te_2']
res['E(V)|X,T = 0'] = pred['Uplift_mean|T=0'] * pred['E(V)|T = 0, D = 1']
res['E(V)|X,T = 1'] = pred['Uplift_mean|T=1'] * pred['E(V)|T = 1, D = 1']
res['Gi'] = res['E(V)|X,T = 1']-res['E(V)|X,T = 0']
#res['Value_mean'] = pred['Value_mean']
res
#res.to_csv('/content/drive/MyDrive/Sinopac/Results/Prediction_results_All_202311.csv')
#1066146 rows × 10 columns

In [ ]:
res[['Gi', 'Value_X']].describe()

In [ ]:
res['Gi'].corr(res['Value_X'])

##Plot

In [ ]:
import pandas as pd
pred = pd.read_csv('/content/drive/MyDrive/Sinopac/Results/Prediction_results_202311.csv', index_col = 0)
res = pd.read_csv('/content/drive/MyDrive/Sinopac/Results/Prediction_results_All_202311.csv', index_col = 0)
res

In [ ]:
# Import libraries
import matplotlib.pyplot as plt
import numpy as np

plt.figure(figsize=(15, 5))

plt.subplot(121)
plt.scatter(res.iloc[:,1], res['X_te_2'])
plt.xlabel('Gene PR')
plt.ylabel('X-Learner(pred)')
plt.subplot(122)
plt.scatter(res.iloc[:,1], res.iloc[:,-1])
plt.xlabel('Gene PR')
plt.ylabel('Gi(pred)')
plt.legend()

In [ ]:
# Import libraries
import matplotlib.pyplot as plt
import numpy as np

plt.figure(figsize =(15, 10))

plt.subplot(141)
plt.boxplot(res['Uplift_mean'], labels=['Uplift_mean'])
# Creating plot
plt.subplot(142)
plt.boxplot(res['X_te_2'], labels=['X-Learner'])
plt.subplot(143)
plt.boxplot(res[['Uplift_mean|T=0', 'Uplift_mean|T=1']], labels=['Uplift|T=0', 'Uplift|T=1'])
plt.subplot(144)
plt.boxplot(res[['E(V)|X,T = 0', 'E(V)|X,T = 1']], labels=['E(V)|T=0', 'E(V)|T=1'])

# show plot
plt.show()

# 名單